In [ ]:
from pygeotemporal.sensors import SensorsApi
from pygeotemporal.streams import StreamsApi
from pygeotemporal.datapoints import DatapointsApi

import matplotlib.pyplot as plt
import pandas as pd

from datetime import datetime
from dateutil.parser import parse

from pprint import pprint


host = r"https://gltg-dev.ncsa.illinois.edu/geostreams"
username = ""
password = ""

output_directory = r"downloads"

sensorclient = SensorsApi(host=host, username=username, password=password)
streamclient = StreamsApi(host=host, username=username, password=password)
datapointclient = DatapointsApi(host=host, username=username, password=password)

In [ ]:
sensor_ids = [8404, 8407]
sensors = []
datapoint_rows = []
all_parameters = []
start_time = None
end_time = None

In [ ]:
for sensor_id in sensor_ids:    
    sensor = sensorclient.sensor_get(sensor_id).json()['sensor']
    
    if sensor['max_end_time'] == 'N/A':
        print("Updating sensor statistics for sensor_id=%s" % sensor_id)
        sensorclient.sensor_statistics_post(sensor_id)

    for parameter in sensor['parameters']:
        if parameter[-2:] == 'qc':
            continue
        if parameter not in all_parameters:
            all_parameters.append(parameter)
            
    # Get maximum time range for all sensors
    if end_time is None or sensor['max_end_time'] > end_time:
        end_time = sensor['max_end_time']
    if start_time is None or sensor['min_start_time'] < start_time:
        start_time = sensor['min_start_time']

    end_time_all = datetime.strptime(end_time[:-4],'%Y-%m-%dT%H:%M')
    start_time_all = datetime.strptime(start_time[:-4],'%Y-%m-%dT%H:%M')


In [ ]:
rows = []
for sensor_id in sensor_ids:    
    datapoints = datapointclient.get_datapoints_by_sensor_id(sensor_id, since='20200101').json()

    for datapoint in datapoints:
        row = []
        row.append(sensor['id'])
        row.append(sensor['name'])
        row.append(sensor['geometry']['coordinates'][1])
        row.append(sensor['geometry']['coordinates'][0])
        row.append(sensor['properties']['huc']['huc_name'])
        row.append(sensor['properties']['huc']['huc8']['code'])
        try:
            row.append(datetime.strptime(datapoint['end_time'][:-4].replace('"T"','T'),'%Y-%m-%dT%H:%M'))
        except:
            pprint(datapoint)
            break
              
        for parameter in all_parameters:
            if parameter in datapoint['properties']:
                row.append(datapoint['properties'][parameter])
            else:
                row.append('')
                
        rows.append(row)
    
sensor_values = ['SENSOR_ID','NAME','LATITUDE','LONGITUDE','HUC NAME','HUC8']
columns = sensor_values + ['datetime'] + all_parameters

df = pd.DataFrame(rows,columns=columns)
for param in all_parameters:
    df[param] = pd.to_numeric(df[param])
df.head(3)

In [ ]:
df[all_parameters].describe()

In [ ]:
for param in all_parameters:
    df.plot(kind='line',x='datetime',y=param)